# Binary Classification Deep Learning Model for [Project Name] Using Keras Version 3
### David Lowe
### November 1, 2019
Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: [Sample Paragraph - The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The Connectionist Bench dataset is a binary classification situation where we are trying to predict one of the two possible outcomes.]

INTRODUCTION: [Sample Paragraph - The data file patterns obtained by bouncing sonar signals off a metal cylinder or a rock at various angles and under various conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock. Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time.]

ANALYSIS: [Sample Paragraph - The baseline performance of the model achieved an average accuracy score of 79.48%. Using the same training parameters, the model processed the test dataset with an accuracy of 82.69%, which was even better than results from the training data.]

CONCLUSION: [Sample Paragraph - For this dataset, the model built using Keras and TensorFlow achieved a satisfactory result and should be considered for future modeling activities]

Dataset Used: [Connectionist Bench (Sonar, Mines vs. Rocks) Data Set]

Dataset ML Model: Binary classification with numerical attributes

Dataset Reference: [https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar,+Mines+vs.+Rocks%29]

One potential source of performance benchmarks: [https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/]

Any deep-learning modeling project genrally can be broken down into about six major tasks:
0. Prepare Environment
1. Load Data
2. Define Model
3. Fit and Evaluate Model
4. Optimize Model
5. Finalize Model

# Section 0. Prepare Environment

In [1]:
# Create the random seed numbers for reproducible results
seedNum = 88

In [2]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [3]:
# Configure a new global `tensorflow` session
import keras as K
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
tf.set_random_seed(seedNum)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

Using TensorFlow backend.


In [4]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the flag for splitting the dataset
splitDataset = True
splitPercentage = 0.25

# Set various default Keras modeling parameters
model_kernel_init = K.initializers.glorot_uniform(seed=seedNum)
model_loss = 'binary_crossentropy'
model_optimizer = 'adam'
model_metrics = ['accuracy']
model_epochs = 100
model_batches = 5

# Set the number of folds for cross validation
folds = 10

In [5]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [6]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Data

In [7]:
if (notifyStatus): email_notify("Phase 1 Load Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [8]:
# Load the dataset
df_original = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header=None)
dataset = df_original.values
print(dataset)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]


In [ ]:
# Split the original dataset into input (X) and output (y) variables
X_original = dataset[:,0:60].astype(float)
y_original = dataset[:,60]
print('Shape of X_original:', X_original.shape, '| Shape of y_original:', y_original.shape)

In [9]:
# Encode class values as integers and perform one-hot-encoding
encoder = LabelEncoder()
encoder.fit(y_original)
y_encoded = encoder.transform(y_original)
print(y_encoded)

Shape of X_original: (768, 8) | Shape of y_original: (768,)


In [10]:
# Split the data further into training and test datasets
if (splitDataset):
    X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, test_size=splitPercentage, random_state=seedNum)
else:
    X_train, y_train = X_original, y_original
    X_test, y_test = X_original, y_original
print('Shape of X_train:', X_train.shape, '| Shape of y_train:', y_train.shape)
print('Shape of X_test:', X_test.shape, '| Shape of y_test:', y_test.shape)

Shape of X_train: (576, 8) | Shape of y_train: (576,)
Shape of X_test: (192, 8) | Shape of y_test: (192,)


In [11]:
if (notifyStatus): email_notify("Phase 1 Load Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Define Model

In [12]:
if (notifyStatus): email_notify("Phase 2 Define Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [13]:
# Define the Keras model required for KerasClassifier
def create_cv_model():
    model = K.models.Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer=default_kernel_init, activation='relu'))
    model.add(Dense(1, kernel_initializer=default_kernel_init, activation='sigmoid'))
    model.compile(loss=default_loss, optimizer=default_optimizer, metrics=default_metrics)
    return model

In [14]:
# Initialize the Keras model
cv_model = KerasClassifier(build_fn=create_cv_model, epochs=model_epochs, batch_size=model_batches, verbose=0)

In [15]:
if (notifyStatus): email_notify("Phase 2 Define Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Fit and Evaluate Model

In [16]:
if (notifyStatus): email_notify("Phase 3 Fit and Evaluate Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [17]:
# Fit and evaluate the Keras model using 10-fold cross validation
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seedNum)
results = cross_val_score(cv_model, X_train, y_train, cv=kfold)
print('Generating results using the metrics of', model_metrics)
print('All cross-Validate results:', results)
print('Average cross-Validate results:', results.mean())






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Generating results using the metrics of ['accuracy']
All cross-Validate results: [0.67796611 0.68965517 0.5862069  0.72413792 0.70689656 0.77192982
 0.77192982 0.73684211 0.63157895 0.70175438]
Average cross-Validate results: 0.6998897739355903


In [18]:
if (notifyStatus): email_notify("Phase 3 Fit and Evaluate Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Optimize Model

In [19]:
if (notifyStatus): email_notify("Phase 4 Optimize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [20]:
# Define the Keras model required for KerasClassifier
def create_grid_model(optimizer, kernel_init):
    model = K.models.Sequential()
    model.add(Dense(60, input_dim=60, kernel_initializer=kernel_init, activation='relu'))
    model.add(Dense(1, kernel_initializer=kernel_init, activation='sigmoid'))
    model.compile(loss=default_loss, optimizer=optimizer, metrics=default_metrics)
    return model

In [21]:
# create model
grid_model = KerasClassifier(build_fn=create_grid_model, verbose=0)

# Perform grid search using different epochs, batch sizes, and optimizers
optimizer_grid = ['rmsprop', 'adam']
init_grid = ['Constant', 'RandomNormal', 'RandomUniform']
epoch_grid = [100, 150, 200]
batch_grid = [5, 10, 15]
param_grid = dict(optimizer=optimizer_grid, kernel_init=init_grid, epochs=epoch_grid, batch_size=batch_grid)
# param_grid = dict(optimizer=optimizer_grid, epochs=epoch_grid, batch_size=batch_grid)
grid = GridSearchCV(estimator=grid_model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [ ]:
best_optimizer = 'rmsprop'
best_kernel_init = 'RandomNormal'
best_epoch = 150
best_batch = 10

In [ ]:
# Create the final model for evaluating the test dataset
model = create_grid_model(best_optimizer, best_kernel_init)
model.fit(X_train, y_train, epochs=best_epoch, batch_size=best_batch, verbose=1)

In [ ]:
# Display a summary of the final model
print(model.summary())

In [ ]:
# Evaluate the Keras model on previously unseen data
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

In [ ]:
if (notifyStatus): email_notify("Phase 4 Optimize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 5. Finalize Model

In [ ]:
if (notifyStatus): email_notify("Phase 5 Finalize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
# Make class predictions with the model
predictions = model.predict_classes(X_original)

# Summarize the first 20 cases
for i in range(20):
	print('%s => %d (expected %d)' % (X_original[i].tolist(), predictions[i], y_original[i]))

In [ ]:
if (notifyStatus): email_notify("Phase 5 Finalize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [ ]:
print ('Total time for the script:',(datetime.now() - startTimeScript))